In [3]:
import json
from shapely import Point
import networkx as nx
from networkx.readwrite import json_graph
from prettytable import PrettyTable
import networkx as nx
from tabulate import tabulate
import matplotlib.pyplot as plt
import os
import rasterio

In [7]:
class GraphOrientationGeometry:
    def __init__(self,dataset, input_Folder, output_folderGeom):
        object.__init__(self)
        self.initVariables(dataset, input_Folder, output_folderGeom)

    def initVariables(self, dataset, input_Folder, output_folderGeom):
        self.input_Folder = input_Folder
        self.output_folderGeom = output_folderGeom
        self.dataset = dataset
        
        self.graphBasedGeometry = None

        self.nodes_path = "Data SIG/" + str(self.dataset) + "/"+self.input_Folder+"/Nodes.json"  # Path to the nodes shapefile
        self.pipes_path = "Data SIG/" + str(self.dataset) + "/"+self.input_Folder+"/Pipes.json"  # Path to the pipes shapefile

        self.nodes_path_out_geom = "Data SIG/" + str(self.dataset) + "/"+self.output_folderGeom+"/Nodes.json"  # Path to the nodes shapefile
        self.pipes_path_out_geom = "Data SIG/" + str(self.dataset) + "/"+self.output_folderGeom+"/Pipes.json"  # Path to the pipes shapefile

        self.mapping_color_components = {

            "Manholes": "red", "Structures": "springgreen", "Pumps": "yellow",
            "Fittings": "cyan", "TreatmentPlant": "black", "Accessories": "orange",
            "Dummy": "violet", "Appariel": "blue", "Deversoir": "bisque", "PosteRefoulement": "peru"
        }
        
    def readJsoneFile(self, file):
        if file == "Nodes":
            # Load the JSON data
            with open(self.nodes_path, 'r') as f:
                self.dataNodes = json.load(f)
                print(f"The file {file} is opned correctly")
                print(f'The number of elements in the {file} is {len(self.dataNodes)}')
        elif file == "Pipes":
            # Load the JSON data
            with open(self.pipes_path, 'r') as f:
                self.dataPipes = json.load(f)
                print(f"The file {file} is opned correctly")
                print(f'The number of elements in the {file} is {len(self.dataPipes)}')
        else:
            print(f"The file name {file} doesnt exist")
    
    def orientationBasedOnGeometry(self):

        self.node_index = {feature['properties']['id']: Point(feature['geometry']['coordinates']) for feature in self.dataNodes}
        self.node_type = {feature['properties']['id']: feature['properties']['source_1'] for feature in self.dataNodes}
        self.graphBasedGeometry = nx.DiGraph()
        for pipe in self.dataPipes:
            #print("*****************************")
            #print(f'The id of the pipe is {pipe["properties"]["id"]}')
            pipeGeom = pipe["geometry"]["coordinates"]
            #print(f'The geometry of the pipe is {pipeGeom}')
            #if pipe["properties"]["id"] != 20260:
            pipeExtremity1 = Point(pipeGeom[0])
            pipeExtremity2 = Point(pipeGeom[-1])

            '''print(f'The first extremity is {pipeExtremity1}')
            print(f'The second extremity is {pipeExtremity2}')'''

            node1Id = pipe["properties"]["sourceNode"]
            node2Id = pipe["properties"]["targetNode"]
            geomNode1Id = self.node_index[node1Id]
            disatnce1 = geomNode1Id.distance(pipeExtremity1)
            distance2 = geomNode1Id.distance(pipeExtremity2)
            if disatnce1 < distance2:
                self.graphBasedGeometry.add_node(node1Id, col =self.mapping_color_components[self.node_type[node1Id]] )
                self.graphBasedGeometry.add_node(node2Id, col =self.mapping_color_components[self.node_type[node2Id]] )
                self.graphBasedGeometry.add_edge(node1Id, node2Id)
            else:
                self.graphBasedGeometry.add_node(node1Id, col =self.mapping_color_components[self.node_type[node1Id]]  )
                self.graphBasedGeometry.add_node(node2Id, col =self.mapping_color_components[self.node_type[node2Id]]  )
                self.graphBasedGeometry.add_edge(node2Id, node1Id)

    

    def saveGraphInJson(self):
        '''data = json_graph.node_link_data(self.graphBasedGeometry)
        with open(self.pipes_path_out_geom, 'w') as file:
            import json
            json.dump(data, file)'''

        with open(self.nodes_path_out_geom, 'w') as file:
            import json
            json.dump(self.dataNodes, file)
        with open(self.pipes_path_out_geom, 'w') as file:
            import json
            json.dump(self.dataPipes, file)

    def describeGraph(self):

        # Number of nodes
        num_nodes = len(self.graphBasedGeometry.nodes())
        print(f"Number of nodes: {num_nodes}")

        # Number of edges
        num_edges = len(self.graphBasedGeometry.edges())
        print(f"Number of edges: {num_edges}")

        self.componentsGraphs = nx.weakly_connected_components(self.graphBasedGeometry)

        subgraphs_info = []

        for component in self.componentsGraphs:
            subgraph = self.graphBasedGeometry.subgraph(component)
            num_nodes = len(subgraph)
            num_edges = subgraph.size()
            subgraphs_info.append((num_nodes, num_edges))

        table_headers = ["Component Number", "Number of Nodes", "Number of Edges"]

        # Prepare the table rows
        table_rows = [(index + 1, num_nodes, num_edges) for index, (num_nodes, num_edges) in enumerate(subgraphs_info)]

        # Print the table using tabulate
        print(tabulate(table_rows, headers=table_headers, tablefmt="grid"))

    def plot_sub_graph_by_id(self, component_number, labeled=False):
        self.componentsGraphs = nx.weakly_connected_components(self.graphBasedGeometry)

        for index, component in enumerate(self.componentsGraphs, start=1):
            if index == component_number:
                subgraph = self.graphBasedGeometry.subgraph(component)
                colorNodes = nx.get_node_attributes(subgraph, 'col')
                colorList = [colorNodes[node] for node in colorNodes]

                plt.figure(3, figsize=(30, 30)) 
                pos = nx.spring_layout(subgraph)

                # Common code for both cases
                if labeled:
                    nx.draw(
                        subgraph,
                        pos=pos,
                        node_size=100,
                        with_labels=True,
                        node_color=colorList,
                        labels={node: node for node in subgraph.nodes()},
                    )
                else:
                    nx.draw(
                        subgraph,
                        pos=pos,
                        node_size=100,
                        with_labels=False,
                        node_color=colorList,
                    )

                # Add edge labels
                #edge_labels = {(edge[0], edge[1]): str(subgraph.edges[edge]['idEdge']) for edge in subgraph.edges}
                nx.draw_networkx_edge_labels(subgraph, pos=pos) if labeled else None

                plt.show()
                break

In [ ]:
class GraphOrientationElevation:
    def __init__(self,dataset, input_Folder, output_folderElev):
        object.__init__(self)
        self.initVariables(dataset, input_Folder, output_folderElev)

    def initVariables(self, dataset, input_Folder, output_folderElev):
        self.input_Folder = input_Folder

        self.output_folderElev = output_folderElev
        self.dataset = dataset

        self.graphBasedElevation = None

        self.nodes_path = "Data SIG/" + str(self.dataset) + "/"+self.input_Folder+"/Nodes.json"  # Path to the nodes shapefile
        self.pipes_path = "Data SIG/" + str(self.dataset) + "/"+self.input_Folder+"/Pipes.json"  # Path to the pipes shapefile


        self.nodes_path_out_elev = "Data SIG/" + str(self.dataset) + "/"+self.output_folderElev +"/Nodes.json"  # Path to the nodes shapefile
        self.pipes_path_out_elev = "Data SIG/" + str(self.dataset) + "/"+self.output_folderElev +"/Pipes.json"  # Path to the pipes shapefile

        self.mapping_color_components = {

            "Manholes": "red", "Structures": "springgreen", "Pumps": "yellow",
            "Fittings": "cyan", "TreatmentPlant": "black", "Accessories": "orange",
            "Dummy": "violet", "Appariel": "blue", "Deversoir": "bisque", "PosteRefoulement": "peru"
        }
        
    def readJsoneFile(self, file):
        if file == "Nodes":
            # Load the JSON data
            with open(self.nodes_path, 'r') as f:
                self.dataNodes = json.load(f)
                print(f"The file {file} is opned correctly")
                print(f'The number of elements in the {file} is {len(self.dataNodes)}')
        elif file == "Pipes":
            # Load the JSON data
            with open(self.pipes_path, 'r') as f:
                self.dataPipes = json.load(f)
                print(f"The file {file} is opned correctly")
                print(f'The number of elements in the {file} is {len(self.dataPipes)}')
        else:
            print(f"The file name {file} doesnt exist")
    

    def saveGraphInJson(self):
        with open(self.nodes_path_out_elev, 'w') as file:
            import json
            json.dump(self.dataNodes, file)
        with open(self.pipes_path_out_elev, 'w') as file:
            import json
            json.dump(self.dataPipes, file)


    def describeGraph(self):

        # Number of nodes
        num_nodes = len(self.graphBasedElevation.nodes())
        print(f"Number of nodes: {num_nodes}")

        # Number of edges
        num_edges = len(self.graphBasedElevation.edges())
        print(f"Number of edges: {num_edges}")

        self.componentsGraphs = nx.weakly_connected_components(self.graphBasedElevation)

        subgraphs_info = []

        for component in self.componentsGraphs:
            subgraph = self.graphBasedElevation.subgraph(component)
            num_nodes = len(subgraph)
            num_edges = subgraph.size()
            subgraphs_info.append((num_nodes, num_edges))

        table_headers = ["Component Number", "Number of Nodes", "Number of Edges"]

        # Prepare the table rows
        table_rows = [(index + 1, num_nodes, num_edges) for index, (num_nodes, num_edges) in enumerate(subgraphs_info)]

        # Print the table using tabulate
        print(tabulate(table_rows, headers=table_headers, tablefmt="grid"))

    def plot_sub_graph_by_id(self, component_number, labeled=False):
        self.componentsGraphs = nx.weakly_connected_components(self.graphBasedElevation)
        for index, component in enumerate(self.componentsGraphs, start=1):
            if index == component_number:
                subgraph = self.graphBasedElevation.subgraph(component)
                colorNodes = nx.get_node_attributes(subgraph, 'col')
                colorList = [colorNodes[node] for node in colorNodes]

                plt.figure(3, figsize=(30, 30)) 
                pos = nx.spring_layout(subgraph)

                # Common code for both cases
                if labeled:
                    nx.draw(
                        subgraph,
                        pos=pos,
                        node_size=100,
                        with_labels=True,
                        node_color=colorList,
                        labels={node: node for node in subgraph.nodes()},
                    )
                else:
                    nx.draw(
                        subgraph,
                        pos=pos,
                        node_size=100,
                        with_labels=False,
                        node_color=colorList,
                    )

                # Add edge labels
                #edge_labels = {(edge[0], edge[1]): str(subgraph.edges[edge]['idEdge']) for edge in subgraph.edges}
                nx.draw_networkx_edge_labels(subgraph, pos=pos) if labeled else None

                plt.show()
                break

    
    def addElevationToNodes(self, folder_path_REGALTI):
        
        files = [f for f in os.listdir(folder_path_REGALTI)]

        for node in self.dataNodes:
            if 'elevation' in node['properties']:
                del node["properties"]["elevation"]

        for asc_file in files:
            # Open the ASC file for reading
            with rasterio.open(os.path.join(folder_path_REGALTI, asc_file)) as src:
                # Extract elevation values for all nodes
                elevations = src.read(1)  # Assuming the elevation values are in the first band
                
                # Get the affine transformation to convert pixel coordinates to map coordinates
                transform = src.transform

                for node in self.dataNodes:
                    node_x, node_y = node['geometry']['coordinates'][0], node['geometry']['coordinates'][1]

                    # Convert node coordinates to pixel coordinates using the ASC file's transformation
                    col, row = ~transform * (node_x, node_y);
                
                    if 0 <= int(row) < src.height and 0 <= int(col) < src.width:
                        node['properties']['elevation'] = float(elevations[int(row), int(col)])
        for node in self.dataNodes:
            if 'elevation' not in node['properties']:
                print("oui")
                node['properties']['elevation'] = -10

        with open(self.nodes_path_out_elev, 'w') as f:
            json.dump(self.dataNodes, f, indent=4)  # Use 'indent' for pretty printing


    def orientationBasedElev(self):
        self.graphBasedElevation = nx.DiGraph()
        mapping_node_elevation = {}
        for node in self.dataNodes:
            mapping_node_elevation[node['properties']['id']] = node['properties']['elevation']
        print(len(mapping_node_elevation))
        for edge in self.dataPipes:
            node1 = edge['properties']['sourceNode']
            node2 = edge['properties']['targetNode']

            cirMode = edge['properties']['modecirc_l']
            if (mapping_node_elevation[node1] > mapping_node_elevation[node2] and cirMode == "Gravitaire") or (mapping_node_elevation[node1] < mapping_node_elevation[node2] and cirMode == "Forcé"):
                self.graphBasedElevation.add_node(node1)
                self.graphBasedElevation.add_node(node2)
                self.graphBasedElevation.add_edge(node1,node2)
            elif (mapping_node_elevation[node1] < mapping_node_elevation[node2] and cirMode == "Gravitaire") or (mapping_node_elevation[node1] > mapping_node_elevation[node2] and cirMode == "Forcé"):
                self.graphBasedElevation.add_node(node1)
                self.graphBasedElevation.add_node(node2)
                self.graphBasedElevation.add_edge(node2,node1)
            else:
                self.graphBasedElevation.add_node(node1)
                self.graphBasedElevation.add_node(node2)
                self.graphBasedElevation.add_edge(node2,node1)
                self.graphBasedElevation.add_edge(node1,node2)

    def getInfor(self):
        bidirectional_edges = set((min(source, target), max(source, target)) for source, target in self.graphBasedElevation.edges() if self.graphBasedElevation.has_edge(target, source))
        print(f"The number of bidirectional arcs: {len(bidirectional_edges)}")

        dic = {}

        for edge in bidirectional_edges:
            print("***************************")
            node1, node2 = edge
            

            all_arcs_with_node1_and_2 = [(source, target) for source, target in self.graphBasedElevation.edges() if (node1 in {source, target} or node2 in {source, target})]
            

            arcs_without_node1_and_2 = [arc for arc in all_arcs_with_node1_and_2 if arc != (node1, node2) and arc != (node2, node1)]

            
            arcs_without_bidirectional = []

            for arc in arcs_without_node1_and_2:
                v1, v2 = arc

                if (v2, v1) not in arcs_without_bidirectional:
                    arcs_without_bidirectional.append(arc)

            direction, status = self.infer_direction(edge, arcs_without_bidirectional)

            if len(arcs_without_bidirectional) == 1:
                if direction in dic:
                    dic[direction] += 1
                else: 
                    dic[direction] = 1
                '''print(f'The edge is {edge}')
                print(f'All arcs {all_arcs_with_node1_and_2}')
                print(f'All arcs without {(node1, node2)} and {(node2, node1)} \n: {arcs_without_node1_and_2}')
                print(f'All arcs without bidirectional \n: {arcs_without_bidirectional}')
                print(direction)'''
            elif len(arcs_without_bidirectional) == 2:
                if status:
                    print(f'The edge is {edge}')
                    #print(f'All arcs {all_arcs_with_node1_and_2}')
                    print(f'All arcs without {(node1, node2)} and {(node2, node1)} \n: {arcs_without_node1_and_2}')
                    print(f'All arcs without bidirectional \n: {arcs_without_bidirectional}')

            

        print(dic)


    def infer_direction(self, edge, arcs):
        direction = True
        status = False
        node1, node2 = edge
        if len(arcs) == 1:
            source, target = arcs[0]

            if node1 == source or node2 == target:
                direction = False
        elif len(arcs) == 2:
            source1, target1 = arcs[0]
            source2, target2 = arcs[1]

            if node1 not in set([source1, source2, target1, target2]) or node2 not in set([source1, source2, target1, target2]):
                status = True
                

        return direction, status

            








